# Data Preparation



In [1]:
import os
import numpy as np
import tensorflow as tf


In [2]:

class MetaDataLoader:
    def __init__(self, data_dir, num_samples_per_location=100):
        self.data_dir = data_dir
        self.num_samples_per_location = num_samples_per_location

    def _load_and_process_data(self, locations):
        data_dict = {}

        for location in locations:
            location_dir = os.path.join(self.data_dir, location)

            train_data_path = os.path.join(location_dir, "train_data.npy")
            train_label_path = os.path.join(location_dir, "train_label.npy")
            vali_data_path = os.path.join(location_dir, "vali_data.npy")
            vali_label_path = os.path.join(location_dir, "vali_label.npy")

            # Load training data and labels
            train_data = np.load(train_data_path)
            train_label = np.load(train_label_path)

            # Load validation data and labels
            vali_data = np.load(vali_data_path)
            vali_label = np.load(vali_label_path)

            # Store data and labels in the data_dict with location as key
            data_dict[location] = {
                'train_data': train_data,
                'train_label': train_label,
                'vali_data': vali_data,
                'vali_label': vali_label
            }

        return data_dict

    def _create_episode(self, locations):

        data_dict = self._load_and_process_data(locations)

        #--------- Create the support set -------------
        selected_data = []
        selected_labels = []
        for location in locations:
            selected_data.extend(data_dict[location]['train_data'][:num_samples_per_location])
            selected_labels.extend(data_dict[location]['train_label'][:num_samples_per_location])


        data = np.array(selected_data)
        labels = np.array(selected_labels)

        # Shuffle the data and labels if needed
        indices = np.arange(data.shape[0])
        np.random.shuffle(indices)
        support_set_data = data[indices]
        support_set_labels = labels[indices]

        #--------- End create the support set -------------

        #--------- Create the query set -------------
        selected_data = []
        selected_labels = []

        for location in locations:
            selected_data.extend(data_dict[location]['vali_data'][:num_samples_per_location])
            selected_labels.extend(data_dict[location]['vali_label'][:num_samples_per_location])

        data = np.array(selected_data)
        labels = np.array(selected_labels)

        # Shuffle the data and labels if needed
        indices = np.arange(data.shape[0])
        np.random.shuffle(indices)
        query_set_data = data[indices]
        query_set_labels = labels[indices]

        #--------- End create the query set -------------

        return support_set_data, support_set_labels, query_set_data, query_set_labels

    def create_multi_episodes(self, num_episodes, locations):
        episodes = []
        for _ in range(num_episodes):
            support_set_data, support_set_labels, query_set_data, query_set_labels = self._create_episode(locations)
            episode = {
                "support_set_data": support_set_data,
                "support_set_labels": support_set_labels,
                "query_set_data": query_set_data,
                "query_set_labels": query_set_labels
            }
            episodes.append(episode)
        return episodes


In [3]:
%cd '/content/drive/MyDrive/Meta_learning_research/Notebooks/'
data_dir = './samples/'  # Replace with the path to your directory containing numpy files
locations_meta_training = ['Alexander', 'Rowancreek']
locations_meta_testing = ['Covington']
num_samples_per_location = 100  # Configure the number of samples per location
num_episodes = 10  # Number of episodes

data_loader = MetaDataLoader(data_dir, num_samples_per_location)

# Create multi episodes for meta-training
episodes = data_loader.create_multi_episodes(num_episodes, locations_meta_training)

/content/drive/MyDrive/Meta_learning_research/Notebooks


In [15]:
episodes[1]['support_set_data'].shape

(200, 224, 224, 8)

# MAML Model


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Define your U-Net architecture
def UNet(input_shape):
    # ... (U-Net implementation details)
    return model

class MemoryModule(tf.keras.layers.Layer):
    def __init__(self, num_classes, memory_size, feature_dim):
        super(MemoryModule, self).__init__()
        self.num_classes = num_classes
        self.memory_size = memory_size
        self.feature_dim = feature_dim

        self.memory = self.add_weight(shape=(num_classes, memory_size, feature_dim),
                                      initializer='random_normal',
                                      trainable=True)

    def call(self, query):
        query_norm = tf.nn.l2_normalize(query, axis=-1)
        memory_norm = tf.nn.l2_normalize(self.memory, axis=-1)
        similarity = tf.matmul(tf.expand_dims(query_norm, 1), memory_norm, transpose_b=True)

        _, indices = tf.math.top_k(similarity, k=1)

        return tf.gather(self.memory, indices)

class MAMLModel(tf.keras.Model):
    def __init__(self, memory_module):
        super(MAMLModel, self).__init__()
        self.memory_module = memory_module
        self.unet = UNet()  # Instantiate your U-Net architecture here

        self.fc = Dense(num_classes)  # Dense layer for classification

    def call(self, x):
        features = self.unet(x)  # Pass input through U-Net

        query = tf.reshape(features, (features.shape[0], -1))  # Flatten

        closest_representations = self.memory_module(query)

        combined_features = closest_representations

        logits = self.fc(combined_features)
        return logits

# Hyperparameters
num_classes = ...
memory_size = ...
feature_dim = ...
# Define other hyperparameters for U-Net and training

# Create memory module
memory_module = MemoryModule(num_classes, memory_size, feature_dim)

# Create MAML model with memory module and U-Net
maml_model = MAMLModel(memory_module)

# Loss function and optimizer
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)

# Training loop
for task in tasks:  # Iterate over tasks (few-shot tasks)
    for task_iteration in range(num_task_iterations):
        for example in task.examples:
            with tf.GradientTape() as tape:
                logits = maml_model(example)
                loss = loss_fn(example.labels, logits)

            gradients = tape.gradient(loss, maml_model.trainable_variables)

            updated_representations = memory_module(query)
            updated_representations -= lr_inner * gradients  # Gradient descent update

            memory_module.update_memory(query, updated_representations)

        with tf.GradientTape() as tape:
            meta_loss = compute_meta_loss(maml_model, task)

        meta_gradients = tape.gradient(meta_loss, maml_model.trainable_variables)
        optimizer.apply_gradients(zip(meta_gradients, maml_model.trainable_variables))
